# Variables

In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''
grid_size = 200


# Setup

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'T4S4R9QXGQNM9D1F',
  'HostId': 'KenjYedQDLTpBP3+KIFg9XicS+tABZTpug2/1rxqK/hId+682+e8TS7eGp/ZG7ngK0ILkQ/8bZ5GHLmQffpqFOftt/KGuunp',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'KenjYedQDLTpBP3+KIFg9XicS+tABZTpug2/1rxqK/hId+682+e8TS7eGp/ZG7ngK0ILkQ/8bZ5GHLmQffpqFOftt/KGuunp',
   'x-amz-request-id': 'T4S4R9QXGQNM9D1F',
   'date': 'Tue, 25 Mar 2025 03:57:22 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-dev-sandbox',
   'CreationDate': datetime.datetime(2025, 2, 7, 23, 18, 12, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1

In [5]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=8,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-03-24 22:57:26,398][INFO    ][coiled] Using existing cluster: 'ils-sara (id: 808133)'
[2025-03-24 22:57:26,400][INFO    ][coiled] Attaching to existing cluster (name: ils-sara, https://cloud.coiled.io/clusters/808133?account=wri-cities-data )


Started a new Dask client on Coiled. Dashboard is available at https://cluster-dkufr.dask.host/qpgfqSlVNtf2bAVE/status


# RUN

In [6]:
import s3fs
import fsspec
import traceback

import os


fs = s3fs.S3FileSystem(anon=False)
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities = [x.split('/')[-1] for x in search_buffer_files]
len(cities)

406

In [6]:
from shapely.ops import linemerge
import dask_geopandas as dgpd
from dask import delayed

@delayed
def compute_partial_union(partition):
    """Compute unary_union for a partition."""
    return partition.unary_union

@delayed
def merge_unions(partial_unions):
    """Merge partial unions into a single MultiLineString or LineString."""
    lines = []
    for geom in partial_unions:
        if geom and not geom.is_empty:
            if geom.geom_type == "LineString":
                lines.append(geom)
            elif geom.geom_type == "MultiLineString":
                lines.extend(geom.geoms)  # Extract individual LineStrings
    
    if len(lines) == 0:
        return None  # Handle empty results
    
    return linemerge(lines)  # Final merge

@delayed
def get_unionized_roads(roads):
    """Efficient computation of unary_union for roads using Dask."""
    partitions = roads.to_delayed()  # This only works if `roads` is a Dask GeoDataFrame
    partial_unions = [compute_partial_union(part) for part in partitions]
    final_union = merge_unions(partial_unions)
    return final_union


In [7]:
import dask_geopandas as dgpd
import pandas as pd
from dask import delayed, compute, visualize
import geopandas as gpd
from dask.diagnostics import ProgressBar
%autoreload
from citywide_calculation import get_utm_crs
from metrics_calculation import calculate_minimum_distance_to_roads_option_B
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
#from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection
from scipy.optimize import fminbound, minimize

@delayed
def get_epsg(city_name):
    search_buffer = f'{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet'
    extent = dgpd.read_parquet(search_buffer)
    geometry = extent.geometry[0].compute()
    epsg = get_utm_crs(geometry)
    print(f'{city_name} EPSG: {epsg}')
    return epsg

'''
@delayed
def load_dataset(path, epsg=None):
    """Load a single parquet dataset"""
    dataset = dgpd.read_parquet(path, npartitions=2)
    if epsg:
        dataset = dataset.set_crs("EPSG:4326", allow_override=True) 
        dataset = dataset.to_crs(epsg=epsg)
    return dataset
'''

def load_dataset(path, epsg=None):
    """Load a single parquet dataset lazily"""
    dataset = dgpd.read_parquet(path, npartitions=2)
    if epsg:
        dataset = dataset.set_crs("EPSG:4326", allow_override=True).to_crs(epsg)
    return dataset

@delayed
def row_count(dgdf):
    """Count the rows in a dataframe"""
    row_count = dgdf.map_partitions(len).compute().sum()

    return row_count


def test_math(input):
    return input + input




%autoreload
from metrics_groupby import metrics

@delayed
def metrics(city_name,YOUR_NAME,grid_size):
    grid_cell_count = 0
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
        'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
        'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }
    # Get EPSG
    epsg = get_epsg(city_name).compute()
    # Load grid
    grid = load_dataset(paths['grid'], epsg=epsg)#.compute()
    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])
    grid['cell_area'] = grid.geometry.area

    cells = grid.index.size
    grid_cell_count += cells

    # Load buildings and perform relevant calculations on it
    buildings = load_dataset(paths['buildings'], epsg=epsg)#.compute()
    buildings['area'] = buildings.geometry.area
    joined_buildings = dgpd.sjoin(buildings, grid, predicate='within')  
    counts_buildings = joined_buildings.groupby('index_right').size()
    grid['n_buildings'] = grid.index.map(counts_buildings).fillna(0).astype(int)
    built_area_buildings = joined_buildings.groupby('index_right')['area'].sum()
    grid['built_area'] = grid.index.map(built_area_buildings).fillna(0).astype(float)

    #total_buildings = row_count(buildings).compute()
    #print(total_buildings)
    # Load roads
    roads = load_dataset(paths['roads'], epsg=epsg)#.compute()
    
    #road_union = roads.unary_union.compute()
    #roads = roads.compute()

    # Load intersections
    intersections = load_dataset(paths['intersections'], epsg=epsg)#.compute()

    intersections_3plus = intersections[intersections.street_count >= 3]
    intersections_4way = intersections[intersections.street_count == 4]

    grid['cell_area_km2'] = grid['cell_area']/1000000.
    
    roads_grid_joined = dgpd.sjoin(roads, grid, predicate='within')
    road_length_km = roads_grid_joined.groupby('index_right')['length'].sum()/1000.
    grid['road_length'] = grid.index.map(road_length_km).fillna(0).astype(float)


    joined_intersections_3plus = dgpd.sjoin(intersections_3plus, grid, predicate='within')
    counts_intersections_3plus = joined_intersections_3plus.groupby('index_right').size()
    grid['intersections_3plus'] = grid.index.map(counts_intersections_3plus).fillna(0).astype(int)

    joined_intersections_4way = dgpd.sjoin(intersections_4way, grid, predicate='within')
    counts_intersections_4way = joined_intersections_4way.groupby('index_right').size()
    grid['intersections_4way'] = grid.index.map(counts_intersections_4way).fillna(0).astype(int)


    grid['m3'] = grid['road_length']/grid['cell_area_km2']
    grid['m4'] = grid['intersections_4way'] / grid['intersections_3plus']
    grid['m5'] =  (1000.**2)*(grid['intersections_4way']/grid['cell_area']) #make sure this is equivalent to the meter calculation

    

    grid['m11'] = 1.0*grid['n_buildings'] / grid['cell_area'] # Building density
    grid['m12'] = grid['built_area'] / grid['cell_area'] # Built area share
    grid['m13'] = grid['built_area'] / grid['n_buildings'] # Average building area

    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}.geoparquet'

    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')

    grid.to_parquet(path)
    return grid_cell_count, path


import time
start_time = time.time()  # Start the timer

cities = ['Nairobi']

for city in cities:
    metrics(city, YOUR_NAME, grid_size=grid_size).compute()

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")


Tasks completed in 22.97 seconds.


In [8]:
import numpy as np
from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection

# Function to calculate the angle between two vectors
def calculate_angle(vector1, vector2):
    angle = np.arctan2(vector2[1], vector2[0]) - np.arctan2(vector1[1], vector1[0])
    angle = np.degrees(angle)
    if angle < 0:
        angle += 360
    return angle

def extract_coords(geometry):
    if isinstance(geometry, (LineString, Polygon)):
        # If it's a single geometry, return its coordinates
        return list(geometry.coords)
    elif isinstance(geometry, (MultiLineString, MultiPolygon)):
        # If it's a multi-part geometry, iterate through the sub-geometries
        coords = []
        for part in geometry.geoms:  # Use .geoms to access each part of the MultiLineString/MultiPolygon
            coords.extend(list(part.coords))
        return coords
    else:
        raise TypeError(f"Unsupported geometry type: {type(geometry)}")

def calculate_sequential_angles_option_A(intersections, roads):
    records = []  # List to store angle records

    # Iterate through each intersection
    for _, intersection in intersections.iterrows():
        intersection_id = intersection['osmid']
        intersection_point = intersection.geometry
        
        # Get all roads connected to the intersection
        connected_roads = roads[(roads['u'] == intersection_id) | (roads['v'] == intersection_id)]
        vectors = []
        
        for _, road in connected_roads.iterrows():
            #coords = list(road.geometry.coords)
            coords = extract_coords(road.geometry)
            
            # Determine the vector for the road segment away from the intersection
            if road['u'] == intersection_id:
                vector = (coords[1][0] - coords[0][0], coords[1][1] - coords[0][1])
            else:
                vector = (coords[-2][0] - coords[-1][0], coords[-2][1] - coords[-1][1])
            
            vectors.append((vector, road['u'], road['v']))

        # Sort vectors based on the angle relative to a fixed axis (e.g., x-axis)
        vectors.sort(key=lambda v: np.arctan2(v[0][1], v[0][0]))

        # Calculate the sequential angles between each pair of vectors
        for i in range(len(vectors)):
            vector1 = vectors[i][0]
            vector2 = vectors[(i + 1) % len(vectors)][0]  # Next vector, looping back to the start
            angle = calculate_angle(vector1, vector2)
            
            record = {
                'Intersection ID': intersection_id,
                'Segment 1': (vectors[i][1], vectors[i][2]),
                'Segment 2': (vectors[(i + 1) % len(vectors)][1], vectors[(i + 1) % len(vectors)][2]),
                'Angle': angle
            }
            records.append(record)

    # Create a DataFrame from the records
    df_angles = pd.DataFrame(records)
    
    return df_angles


city_name = 'Nairobi'

paths = {
    'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
    'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
    'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
    'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
}

epsg = get_epsg(city_name).compute()

roads = load_dataset(paths['roads'], epsg=epsg)#.compute()
intersections = load_dataset(paths['intersections'], epsg=epsg)#.compute()
grid = load_dataset(paths['grid'], epsg=epsg)

# Compute intersection‑level angles (in memory)
intersections_df = intersections.compute()
roads_df         = roads.compute()
grid_df          = grid.compute()

# Build full angles DataFrame
df_angles = calculate_sequential_angles_option_A(intersections_df, roads_df)

# Merge with street_count, then apply your “smallest angles” logic
intersection_meta = intersections_df[['osmid','street_count','geometry']].set_index('osmid')
angles_merged = intersection_meta.merge(
    df_angles.set_index('Intersection ID'),
    left_index=True, right_index=True,
    how='inner'
)

# 3‑way → keep smallest; 4+ → keep two smallest
keep3 = angles_merged[angles_merged.street_count == 3].groupby(level=0).Angle.min()
keep4 = angles_merged[angles_merged.street_count >= 4].groupby(level=0).apply(lambda g: g.nsmallest(2, 'Angle')).reset_index(level=0, drop=True)

kept = pd.concat([keep3, keep4]).reset_index().rename(columns={'index':'osmid','Angle':'Angle'})
kept['angle_diff'] = (90 - kept['Angle']).abs()

# Turn back into a GeoDataFrame
kept_gdf = gpd.GeoDataFrame(
    kept.merge(intersections_df[['osmid','geometry']], on='osmid'),
    geometry='geometry', crs=intersections_df.crs
)

# Spatially join to grid & compute m10
joined_m10 = gpd.sjoin(kept_gdf, grid, predicate='within')
m10_by_cell = joined_m10.groupby('index_right')['angle_diff'].mean()
#grid['m10'] = grid.index.map(m10_by_cell).fillna(0).astype(float)


KeyError: "None of ['Intersection ID'] are in the columns"

In [ ]:
# Compute a small in‑memory map from intersection ID → geometry
ints = intersections.compute()[['osmid','geometry']].set_index('osmid')['geometry']
geom_map = ints.to_dict()

# Partition‑wise tortuosity calculation
def add_tortuosity(df):
    # Safely compute straight‑line distance; avoid divide-by-zero
    df['euclid_dist'] = df.apply(
        lambda r: geom_map.get(r.u).distance(geom_map.get(r.v)) if (r.u in geom_map and r.v in geom_map) else None,
        axis=1
    )
    df['tortuosity'] = df['length'] / df['euclid_dist']
    return df

roads = roads.map_partitions(
    add_tortuosity,
    meta=roads._meta.assign(euclid_dist=float, tortuosity=float)
)

# 3️⃣ Spatial join and aggregate into your grid
roads_grid = dgpd.sjoin(roads, grid, predicate='within')
mean_tortuosity = roads_grid.groupby('index_right')['tortuosity'].mean()
grid['m10'] = grid.index.map(mean_tortuosity).fillna(0).astype(float)


TypeError: float() argument must be a string or a real number, not 'GeoDataFrame'

In [ ]:
# Sum the total number of grid cells
total_grid_cells = sum([grid_cells for grid_cells, path in calculated_grids])
print(f'Total grid cells: {total_grid_cells}')

Total grid cells: 462160


In [12]:
client.close()